In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Process Data

In [2]:
#!/home/lgunthermann/miniconda3/envs/discushack22/bin/python3 process_csv.py

In [3]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

In [4]:
df = pd.DataFrame().reindex(columns=data_coding.columns)
df = df.rename(columns={"Expense Category": "Expense Category (Coding)"})
df.insert(1,'ECRef',[])
df.insert(6,'Expense Category (Invoice)',[])
df.drop(['ID'], axis=1)

,ECRef,Supplier,Party,Total Spend,Expense Category (Coding),Expense Category (Invoice),Transport,Miscellaneous,Catering,Accommodation,...,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking


# Output Dataframe

The new dataframe consists of 64 columns. 
- ECRef linking the entry to the invoice list
- Supplier
- Party
- Total Spend
- Expense Category (According to the coding list)
- Expense Category (According to the invoice list)
- 58 expense categorisations

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def print_positions(party,supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['RegulatedEntityName'] == party) & (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Party'] == party) & (data_coding['Supplier'] == supplier) ])
    
def print_supplier(supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Supplier'] == supplier) ])
    
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

In [6]:
#display(data_coding)

# Data Preview

In [7]:
data_coding.head()

,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,0,Abi Bell,Liberal Democrats,1936.07,Transport,0.0,0.0,0.0,0.0,0.0,1936.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Abigail Baker,Liberal Democrats,748.80,Transport,0.0,0.0,0.0,0.0,0.0,748.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Adam Prices,Plaid Cymru,275.85,Transport,0.0,0.0,0.0,0.0,0.0,275.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adam Williams,Liberal Democrats,484.06,Transport,0.0,0.0,0.0,0.0,0.0,484.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Addison Lee,Conservative Party,22648.35,Transport,0.0,0.0,0.0,0.0,0.0,22648.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data_invoice.head()

,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,AmountInScotland,AmountInWales,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
0,SP0555253,Plaid Cymru,UK Parliamentary general election 12/12/2019,Political Party,200.40,03/01/2019,Unsolicited material to electors,Low Cost Printing,"Precision House, Pentrpoeth Rd, Swansea, SA6 6AA, United Kingdom",£0.00,£0.00,£200.40,£0.00,NaN,03/01/2019,74031,77,2276,NaN,NaN,NaN,False
1,SP0541026,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,60.00,01/04/2019,Advertising,Rayleigh Town Council,"The Pavilion, KGV Playing Field, Bull Lane, Rayleigh, SS6 8JD, United Kingdom",£60.00,£0.00,£0.00,£0.00,01/04/2019,10/05/2019,0,90,68727,NaN,NaN,NaN,False
2,SP0541025,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,36.00,17/06/2019,Market research/canvassing,Rochford District Council,"Council Offices, South Street, ROCHFORD, SS4 1BW, United Kingdom",£36.00,£0.00,£0.00,£0.00,17/06/2019,04/07/2019,0,90,76767,NaN,NaN,NaN,False
3,SP0541024,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9.99,07/01/2019,Advertising,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United Kingdom",£9.99,£0.00,£0.00,£0.00,07/01/2019,10/01/2019,0,90,76765,NaN,NaN,NaN,False
4,SP0541023,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,72.00,18/01/2019,Unsolicited material to electors,Off-Tech,"1 Brodie Road, ENFIELD, ZZ0 0ZZ, United Kingdom",£72.00,£0.00,£0.00,£0.00,18/01/2019,18/02/2019,0,90,76766,NaN,NaN,NaN,False


# Solver

In [34]:
def get_totals(invoice_amounts,total_values,prediction):
    totals = np.zeros_like(total_values)
    for amount, cat in zip(invoice_amounts,prediction):
        totals[cat] += amount
    return totals

def get_error(invoice_amounts,total_values,prediction,loss='MSE',return_totals=False,return_diff=False):
    diff = total_values - get_totals(invoice_amounts,total_values,prediction)
    if loss == 'MSE':
        error = np.square(diff)
    else:
        error = np.abs(diff)
        
    r = [np.mean(error)]
    if return_totals: r+= [totals]
    if return_diff: r+= [diff]
    return r
    
def random_solver(invoice_amounts,total_values,loss='MSE',max_iter=5,max_waiting_iter=1):
    factor = invoice_amounts.shape[0] ** (len(total_values)-1)
    max_iter = int(max_iter * factor)
    max_waiting_iter = int(max_waiting_iter * factor)
    pred = np.random.randint(low=0, high=len(total_values), size=invoice_amounts.shape[0], dtype=int)
    if len(total_values) <= 1:
        return pred

    error, diff = get_error(invoice_amounts,total_values,pred,loss,return_diff=True)
    wait = 0
    visited = []
    while True:   
    
        if wait >= max_waiting_iter or error < 0.01:
            print("Break iter",it)
            break
        
        print(f"{error=} {diff=}")
        cat = np.argmax(diff)
        print([i for i in range(len(pred)) if pred[i]==cat])
        idx = np.random.choice([i for i in range(len(pred)) if pred[i]==cat], size=1)
        print(cat)
        
        error_, diff_, cat_ = error, diff, cat
        for c in range(len(total_values)):
            #if c==cat: continue
            pred[idx] = c
            e, d = get_error(invoice_amounts,total_values,pred,loss,return_diff=True)
            print(f"{c=} {e=}")
            if e<error_:
                error_, diff_, cat_ = e, d, c
                
        if error_ < error:
            wait = 0
            print("Improvement!")
        else: wait+=1
                
        pred[idx] = cat_
        error, diff = error_, diff_
            
    return pred

In [38]:
parties = pd.concat((data_coding['Party'], data_invoice['RegulatedEntityName'])).unique()
categories = data_coding.columns[5:]

error_acc = 0
row_list = []

for party in parties:
    party_coding = data_coding[ data_coding['Party']==party ]
    party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    suppliers = pd.concat(( party_coding['Supplier'], party_invoice['SupplierName'] )).unique()
    
    for supplier in suppliers:
        sup_coding = party_coding[ party_coding['Supplier']==supplier ]
        sup_invoice = party_invoice[ party_invoice['SupplierName']==supplier ]
        
        sup_cats = {}
        
        for cat in categories:
            val = np.sum(sup_coding[cat].values)
            if val >= 0.01:
                sup_cats[cat] = val
        
        if len(sup_cats) < 1:
            continue
        
        spent = np.round(sup_coding['Total Spend'].sum(),2)
        filed = np.round(sup_invoice['TotalExpenditure'].sum(),2)
        
        print(f"{party} - {supplier}")
        print(f"Diff = {abs(spent-filed)} ({spent=} | {filed=}) accross {len(sup_cats)} categories with {sup_coding.shape[0]} codings and {sup_invoice.shape[0]} invoices.")
        
        # Estimate a proper matching if there's more than one invoice
        # random encoding which invoice belongs to which category
        if sup_invoice.shape[0] > 1:
            values = np.fromiter(sup_cats.values(), dtype=float)
            invoices = sup_invoice['TotalExpenditure'].values
            prediction = random_solver(invoices,values,loss='MSE')

            error = get_error(invoices,values,prediction,loss='MAE')[0]
            error_acc += error

            for i, (idx, row) in enumerate(sup_invoice.iterrows()):
                val = row['TotalExpenditure']
                entry = {
                    'ECRef': row['ECRef'],
                    'Supplier': supplier,
                    'Party': party,
                    'Total Spend': val,
                    'Expense Category (Coding)': sup_coding['Expense Category'].mode(),
                    'Expense Category (Invoice)': row['ExpenseCategoryName'],
                    **{k:0 for k in categories}
                        }

                cat = get_nth_key(sup_cats, n=prediction[i])
                entry[cat]=val

                row_list.append(entry)
        else:
            row = sup_invoice.iloc[0]
            entry = {
                'ECRef': row['ECRef'],
                'Supplier': supplier,
                'Party': party,
                'Total Spend': row['TotalExpenditure'],
                'Expense Category (Coding)': sup_coding['Expense Category'].mode(),
                'Expense Category (Invoice)': row['ExpenseCategoryName'],
                **{k:0 for k in categories}
                    }
            entry.update(sup_cats)
            row_list.append(entry)
            
            
        if False:
            print(f"{party} - {supplier}")
            print(f"Diff = {abs(spent-filed)} ({spent=} | {filed=}) accross {len(sup_cats)} categories with {sup_coding.shape[0]} codings and {sup_invoice.shape[0]} invoices.")

            print("\nCoding:")
            display(sup_coding)

            print("\nInvoice:")
            display(sup_invoice)
            
df = pd.DataFrame.from_dict(row_list)

Liberal Democrats - Abi Bell
Diff = 0.0 (spent=1936.07 | filed=1936.07) accross 1 categories with 1 codings and 41 invoices.
Liberal Democrats - Abigail Baker
Diff = 0.0 (spent=4298.8 | filed=4298.8) accross 1 categories with 2 codings and 15 invoices.
Liberal Democrats - Adam Williams
Diff = 0.0 (spent=484.06 | filed=484.06) accross 1 categories with 1 codings and 12 invoices.
Liberal Democrats - Addison Lee
Diff = 23.88000000000011 (spent=2808.24 | filed=2832.12) accross 1 categories with 1 codings and 7 invoices.
Liberal Democrats - Airbnb
Diff = 20.0 (spent=6110.62 | filed=6130.62) accross 2 categories with 2 codings and 28 invoices.
error=7055392.192399998 diff=array([-2666.18,  2646.18])
[1, 2, 7, 9, 13, 14, 17, 19, 22, 23, 24, 25, 27]
1
c=0 e=7459444.192399998
c=1 e=7055392.192399998
error=7055392.192399998 diff=array([-2666.18,  2646.18])
[1, 2, 7, 9, 13, 14, 17, 19, 22, 23, 24, 25, 27]
1
c=0 e=8301298.192399997
c=1 e=7055392.192399998
error=7055392.192399998 diff=array([-2666.

ValueError: 'a' cannot be empty unless no samples are taken

In [ ]:
print("Error:",error)
df.head()

In [39]:
print_positions(party='Liberal Democrats',supplier='Europcar')

Invoices:


,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,AmountInScotland,AmountInWales,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
7345,SP0529573,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,479.69,02/08/2019,Transport,Europcar,"James House, 55 Welford Road, Leicester, LE2 7AR, United Kingdom",£408.25,£46.32,£25.12,£0.00,02/08/2019,02/08/2019,69741,90,75519,NaN,NaN,NaN,False
7346,SP0529572,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,247.12,02/08/2019,Transport,Europcar,"James House, 55 Welford Road, Leicester, LE2 7AR, United Kingdom",£210.32,£23.86,£12.94,£0.00,02/08/2019,02/08/2019,69740,90,75519,NaN,NaN,NaN,False


Sum TotalExpenditure: 726.81

Coding:


,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
64,64,Europcar,Liberal Democrats,726.81,Transport,247.12,0.0,0.0,0.0,0.0,479.69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
